In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import joblib
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
# Define paths for models and datasets
models_dir = "/content/drive/MyDrive/models"
test_data_path = "/content/drive/MyDrive/test_data.csv"
train_data_path = "/content/drive/MyDrive/train_data.csv"

In [4]:
# Load test and train data
test_df = pd.read_csv(test_data_path)
train_df = pd.read_csv(train_data_path)

In [5]:
# Extract features and target variable
X_test = test_df.drop(columns=['Class'])  # Adjust 'Class' column name if different
y_test = test_df['Class']
X_train = train_df.drop(columns=['Class'])
y_train = train_df['Class']

In [6]:
# Dictionary containing model names and their corresponding file paths
model_filenames = {
    "Logistic Regression": "logistic_regression.pkl",
    "Random Forest": "random_forest.pkl",
    "Support Vector Machine": "svm.pkl",
    "XGBoost": "xgboost.pkl",
    "K-Nearest Neighbors": "knn.pkl"
}

In [7]:
# Initialize list to store evaluation results
results = []


In [8]:
# Iterate through models, load them, and evaluate
for model_name, filename in model_filenames.items():
    model_path = os.path.join(models_dir, filename)

    if os.path.exists(model_path):
        # Load the saved model
        model = joblib.load(model_path)

        # Predict on train and test datasets
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Compute evaluation metrics
        metrics = {
            'Model': model_name,
            'Train Accuracy': accuracy_score(y_train, y_train_pred),
            'Test Accuracy': accuracy_score(y_test, y_test_pred),
            'Precision': precision_score(y_test, y_test_pred, zero_division=1),
            'Recall': recall_score(y_test, y_test_pred, zero_division=1),
            'F1 Score': f1_score(y_test, y_test_pred, zero_division=1)
        }
        results.append(metrics)
    else:
        print(f"Model file not found: {model_path}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:41:14] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [9]:
# Convert results to DataFrame and display
evaluation_df = pd.DataFrame(results)
print(evaluation_df)

                    Model  Train Accuracy  Test Accuracy  Precision    Recall  \
0     Logistic Regression        0.979382       0.977797   0.971469  0.893006   
1           Random Forest        0.999300       0.998262   0.998798  0.990761   
2  Support Vector Machine        0.988841       0.988278   0.983868  0.945162   
3                 XGBoost        1.000000       0.999652   0.997918  1.000000   
4     K-Nearest Neighbors        0.999238       0.998973   0.993878  1.000000   

   F1 Score  
0  0.930586  
1  0.994763  
2  0.964126  
3  0.998958  
4  0.996930  


In [10]:
# Save evaluation results to CSV
evaluation_df.to_csv(os.path.join(models_dir, 'evaluation_results.csv'), index=False)